* Weather resource: https://www.timeanddate.com/weather/@2176947
* Webscraping resource: https://stackoverflow.com/questions/51756775/scraping-table-from-website-timeanddate-com
* Install Chromedriver: https://chromedriver.chromium.org/downloads

In [2]:
import pandas as pd
import requests, re, typing
from bs4 import BeautifulSoup as soup
import contextlib
from selenium import webdriver #need to install chromedriver to iterate through day filter
from datetime import datetime

c:\Users\Carl\Anaconda3\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [3]:
def _remove(d:list) -> list:
    return list(filter(None, [re.sub('\xa0', '', b) for b in d]))

In [4]:
@contextlib.contextmanager
def get_weather_data(url:str, by_url = True) -> typing.Generator[dict, None, None]:
   d = soup(requests.get(url).text if by_url else url, 'html.parser')
   _table = d.find('table', {'id':'wt-his'})
   _data = [[[i.text for i in c.find_all('th')], *[i.text for i in c.find_all('td')]] for c in _table.find_all('tr')]
   [h1], [h2], *data, _ = _data
   _h2 = _remove(h2)
   yield {tuple(_remove(h1)):[dict(zip(_h2, _remove([a, *i]))) for [[a], *i] in data]}

In [5]:
def get_weather_dict(chrome_dir, url):
    d = webdriver.Chrome(chrome_dir)
    d.get(url)
    _d = {}
    for i in d.find_element_by_id('wt-his-select').find_elements_by_tag_name('option'):
        i.click()
        with get_weather_data(d.page_source, False) as weather:
            _d[i.text] = weather
    d.close()
    return _d

In [6]:
def get_daily_humidty(_d):
    df_all = pd.DataFrame()
    for date, dict in _d.items():
        for day in dict.values():
            df = pd.DataFrame(day)
            df = df[df["Barometer"]!="N/A"]
            df = df[~df["Barometer"].str.contains("mbar")]
            df["Humidity"] = df["Barometer"].str.split('%', expand=True)[0]
            df["Humidity"] = df["Humidity"].astype(int) / 100
            d = {
                "DATE":[date], 
                "HUMIDITYAVE":[df["Humidity"].mean()],
                "HUMIDITYMIN":[df["Humidity"].min()],
                "HUMIDITYMAX":[df["Humidity"].max()],
            }
            df_day = pd.DataFrame(d)
            df_all = df_all.append(df_day)
    return df_all

In [7]:
this_year = int(datetime.today().strftime('%Y'))
this_month = int(datetime.today().strftime('%m')) 

In [8]:
start_year = 2022
end_year = 2022
df_summary = pd.DataFrame()
for year in range(start_year, end_year+1):
    if year > this_year: break
    for month in range(1, 13):
        if (year == this_year) & (month > this_month): break
        print(year, month)
        url = f'https://www.timeanddate.com/weather/@2176947/historic?month={month}&year={year}'
        chrome_dir = r'C:\Users\Carl\AppData\Local\Temp\7zO4CF7F84B\chromedriver' #insert path to installed chromedriver (exe file)
        _d = get_weather_dict(chrome_dir, url)
        df = get_daily_humidty(_d)
        df_summary = df_summary.append(df) 

2022 5


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="wt-his-select"]"}
  (Session info: chrome=111.0.5563.65)


In [9]:
df_summary 

,DATE,HUMIDITYAVE,HUMIDITYMIN,HUMIDITYMAX
0,1 January 2010,0.675870,0.46,0.89
0,2 January 2010,0.675870,0.46,0.89
0,3 January 2010,0.735918,0.46,1.00
0,4 January 2010,0.832766,0.69,1.00
0,5 January 2010,0.706042,0.51,0.94
0,6 January 2010,0.664490,0.51,0.83
0,7 January 2010,0.827111,0.74,0.94
0,8 January 2010,0.706667,0.54,0.88
0,9 January 2010,0.666596,0.51,0.78
0,10 January 2010,0.563958,0.34,0.83


In [10]:
df_summary.to_csv("humidity - 2022.csv")